<a href="https://colab.research.google.com/github/vol1ura/parkrun_analysis/blob/master/clubmates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random
import re
import requests
import time
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', None)

In [2]:
%pip install random_user_agent

     |████████████████████████████████| 8.2 MB 7.0 MB/s 


In [3]:
from random_user_agent.user_agent import UserAgent

user_agent_rotator = UserAgent()

In [4]:
club_id = 23212 # Wake&Run
home_parkrun = 'Kuzminki'

In [5]:
headers = {
    'Host': 'www.parkrun.ru',
    'User-Agent': user_agent_rotator.get_random_user_agent(),
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-GPC': '1',
    'TE': 'Trailers'
}

In [6]:
page_all_results = requests.get('https://www.parkrun.ru/results/courserecords/', headers=headers)
data = pd.read_html(page_all_results.text)[0]
russian_parkruns = data[data.columns[0]]

In [7]:
dfs = []
for parkrun in tqdm(russian_parkruns):
    time.sleep(3 + 5*random.random())
    parkrun_trim = re.sub(r'[\s-]', '', parkrun)
    url = f'https://www.parkrun.ru/{parkrun_trim}/results/clubhistory/?clubNum={club_id}'
    headers['User-Agent'] = user_agent_rotator.get_random_user_agent()
    club_results = requests.get(url, headers=headers)
    try:
        df = pd.read_html(club_results.text)[0]
        dfs.append(df[df.columns[0]])
        if parkrun == home_parkrun:
            home_list = df[df.columns[0]]
    except:
      print('ОШИБКА - операция завершилась досрочно. Паркран временно заблокировал IP.')
      break

  0%|          | 0/108 [00:00<?, ?it/s]

In [8]:
def last_name_first(full_name: str):
    names = full_name.split()
    last_name = names.pop(1).capitalize()
    names.insert(0, last_name)
    return ' '.join(names)

In [9]:
full_list = pd.concat(dfs).drop_duplicates(keep='last')
outer_home_list = pd.concat([full_list, home_list]).apply(last_name_first).sort_values().drop_duplicates(keep=False).reset_index(drop=True)

### Одноклубники, не бегавшие на домашнем паркране:

In [10]:
outer_home_list.shift(1, fill_value='________Фамилия_Имя___')

0    ________Фамилия_Имя___
1         Мелеховец Василий
2           Пантюхин Андрей
3             Степанов Ника
4             Супонева Инна
5         Харченко Владимир
6           Ширинкин Максим
Name: (Unnamed: 0_level_0, Участник), dtype: object

### Полный список одноклубников:

In [11]:
full_list.apply(last_name_first).sort_values().reset_index(drop=True).shift(1, fill_value='________Фамилия_Имя___')

0      ________Фамилия_Имя___
1               Agievich Ivan
2             Gamzinov Sergey
3             Gamzinova Irina
4             Ganina Victoria
5              Nagibin Evgeny
6        Papodopoulos Ganikus
7              Pominova Elena
8             Savova Veselina
9           Suvorov Aleksandr
10             Timofeeva Yana
11                Voynov Egor
12     Абдурашидов Гаджимурад
13              Акимова Ольга
14            Андрюшенко Глеб
15         Андрюшенко Татьяна
16           Ардабьев Дмитрий
17               Артюшов Иван
18           Бабаева Кристина
19           Бабушинов Сергей
20              Баташева Ника
21              Батяева Дарья
22              Берг Светлана
23            Березина Оксана
24            Березной Степан
25          Бусалаева Евгения
26          Быковский Дмитрий
27              Вавилина Анна
28           Варнавский Павел
29              Вегель Сергей
30         Веденеева Кристина
31            Верялин Василий
32           Верялина Наталия
33        